In [5]:
#imports
import nltk
nltk.download('brown')
from nltk.corpus import brown

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import pyLDAvis
import pyLDAvis.sklearn
import pyLDAvis.gensim_models

import spacy
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, TruncatedSVD, LatentDirichletAllocation

from spacy.lang.en.stop_words import STOP_WORDS as stopwords

from collections import Counter, defaultdict

nlp = spacy.load('en_core_web_sm')

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package brown to
[nltk_data]     /Users/georgegarcia/nltk_data...
[nltk_data]   Package brown is already up-to-date!


Topic Modeling

In [7]:
#directory locations
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
processed_folder = parent_directory + '/data/wine-com/processed/'

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


LoadData

In [8]:
df = pd.read_csv(processed_folder + '1676752918.122732.txt', sep = '|')

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
df.head()

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,url,name,variety,origin,type,description,reviews
0,https://www.wine.com/product/domaine-du-terme-...,Domaine du Terme Gigondas 2019,Rhone Red Blends,"from Gigondas, Rhone, France",Red Wine,NaN,spectacular gigondas wine red cherry flavor so...
1,https://www.wine.com/product/viticcio-chianti-...,Viticcio Chianti Classico Riserva 2017,Sangiovese,"from Chianti Classico, Chianti, Tuscany, Italy",Red Wine,intense ruby red violet nuance complexnose sou...,supple vibrant concentrate feature black cherr...
2,https://www.wine.com/product/bodegas-urbina-cr...,Bodegas Urbina Crianza 2012,Tempranillo,"from Rioja, Spain",Red Wine,medium crimson color emit aromatic array cedar...,dark amber rim red aromas red currant cherry p...
3,https://www.wine.com/product/sanford-sanford-a...,Sanford Sanford & Benedict Vineyard Pinot Noir...,Pinot Noir,"from Sta. Rita Hills, Santa Barbara, Central ...",Red Wine,2017 pinot noir sanford benedict bright beauti...,lastly 2017 pinot noir sanford benedict come s...
4,https://www.wine.com/product/san-simeon-estate...,San Simeon Estate Reserve Petite Sirah 2019,Petite Sirah,"from Paso Robles, Central Coast, California",Red Wine,aromas blackberry ripe plum thyme greet nose a...,cassis cola cream black cherry sauce aroma ric...


In [6]:
# This function comes from the BTAP repo.
def display_topics(model, features, no_top_words=5):
    for topic, words in enumerate(model.components_):
        total = words.sum()
        largest = words.argsort()[::-1] # invert sort order
        print("\nTopic %02d" % topic)
        for i in range(0, no_top_words):
            print("  %s (%2.2f)" % (features[largest[i]], abs(words[largest[i]]*100.0/total)))

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
# Check if the 'category' column exists in the DataFrame
if 'category' in df.columns:
    # Get a list of all unique categories in the DataFrame
    categories = df['category'].unique()

    # Loop through each category and print the number of articles in the DataFrame for that category
    for category in categories:
        num_articles = len(df[df['category'] == category])
        print(f"For {category} we have {num_articles} articles.")
else:
    print("The 'category' column does not exist in the DataFrame.")

The 'category' column does not exist in the DataFrame.


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


what type of wine in the df

In [12]:
# Check if the 'type' column exists in the DataFrame
if 'type' in df.columns:
    # Get a list of all unique types in the DataFrame
    types = df['type'].unique()

    # Loop through each type and print the number of articles in the DataFrame for that type
    for article_type in types:
        num_articles = len(df[df['type'] == article_type])
        print(f"For {article_type} we have {num_articles} articles.")
else:
    print("The 'type' column does not exist in the DataFrame.")

For Red Wine we have 7849 articles.
For White Wine we have 3800 articles.
For Sparkling & Champagne we have 906 articles.
For Pink and Rosé we have 415 articles.
For Collectible we have 13 articles.
For Green Wine we have 10 articles.
For Screw Cap we have 37 articles.
For Boutique we have 12 articles.


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


what review topics of wine in the df

In [13]:
# Check if the 'reviews' column exists in the DataFrame
if 'reviews' in df.columns:
    # Count the number of non-null values in the 'reviews' column
    num_reviews = df['reviews'].count()

    # Print the total number of reviews in the DataFrame
    print(f"There are {num_reviews} reviews in the DataFrame.")
else:
    print("The 'reviews' column does not exist in the DataFrame.")

There are 6643 reviews in the DataFrame.


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
# Define the wine types to search for
wine_types = ['red', 'white', 'sparkling', 'champagne']

# Define empty lists to hold the category, id, text, and review data for each wine type
category_list = []
id_list = []
text_list = []
review_list = []

# Loop through each wine type
for wine_type in wine_types:
    # Filter the DataFrame to select only the rows that match the wine type
    filtered_df = df[df['type'].str.contains(wine_type, case=False)]

    # Loop through each row in the filtered DataFrame
    for index, row in filtered_df.iterrows():
        # Add the category, id, text, and review data to their respective lists
        category_list.append(wine_type)
        id_list.append(row['name'])
        text_list.append(row['description'])
        review_list.append(row['reviews'])

# Create a new DataFrame from the category, id, text, and review lists
wine_df = pd.DataFrame({
    'category': category_list,
    'id': id_list,
    'text': text_list,
    'reviews': review_list
})

# Print the shape of the new DataFrame
print(wine_df.shape)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(13461, 4)


In [17]:
wine_df.head()

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,category,id,text,reviews
0,red,Domaine du Terme Gigondas 2019,NaN,spectacular gigondas wine red cherry flavor so...
1,red,Viticcio Chianti Classico Riserva 2017,intense ruby red violet nuance complexnose sou...,supple vibrant concentrate feature black cherr...
2,red,Bodegas Urbina Crianza 2012,medium crimson color emit aromatic array cedar...,dark amber rim red aromas red currant cherry p...
3,red,Sanford Sanford & Benedict Vineyard Pinot Noir...,2017 pinot noir sanford benedict bright beauti...,lastly 2017 pinot noir sanford benedict come s...
4,red,San Simeon Estate Reserve Petite Sirah 2019,aromas blackberry ripe plum thyme greet nose a...,cassis cola cream black cherry sauce aroma ric...


In [21]:
wine_df['reviews'] = wine_df['reviews'].fillna('')

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
count_text_vectorizer = CountVectorizer(stop_words=stopwords, min_df=5, max_df=0.7)
count_text_vectors = count_text_vectorizer.fit_transform(wine_df["reviews"])
count_text_vectors.shape

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(13461, 4409)

In [23]:
tfidf_text_vectorizer = TfidfVectorizer(stop_words=stopwords, min_df=5, max_df=0.7)
tfidf_text_vectors = tfidf_text_vectorizer.fit_transform(wine_df["reviews"])
tfidf_text_vectors.shape

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(13461, 4409)

## Fitting a Non-Negative Matrix Factorization Model

In [29]:
nmf_text_model = NMF(n_components=5, random_state=314)
W_text_matrix = nmf_text_model.fit_transform(tfidf_text_vectors)
H_text_matrix = nmf_text_model.components_

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [30]:
display_topics(nmf_text_model, tfidf_text_vectorizer.get_feature_names())


Topic 00
  tannin (2.11)
  cherry (1.80)
  red (1.50)
  medium (1.39)
  finish (1.32)

Topic 01
  taste (5.00)
  commentary (2.80)
  san (2.76)
  francisco (2.75)
  wine (2.55)

Topic 02
  lemon (1.47)
  apple (1.41)
  white (1.39)
  fresh (1.17)
  acidity (1.03)

Topic 03
  barrel (1.24)
  sample (1.11)
  wine (1.07)
  fruit (0.85)
  year (0.69)

Topic 04
  cabernet (4.74)
  sauvignon (2.98)
  merlot (1.87)
  franc (1.65)
  petit (1.30)


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## Fitting an LDA Model

In [24]:
# Fit your LDA model here
from sklearn.decomposition import LatentDirichletAllocation

lda_text_model = LatentDirichletAllocation(n_components = 10, random_state=42)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
#creating the matrix 
W_lda_text_matrix = lda_text_model.fit_transform(count_text_vectors)
H_lda_text_matrix = lda_text_model.components_

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [26]:
#saving the matrix of the new topic 
wine_df["lda_topic"] = np.argmax(W_lda_text_matrix, axis = 1).astype(str)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [28]:
#for loop that count the unique topic 
for label in wine_df['lda_topic'].unique():
    print(f"topic {label} observes original label counts of:")
    print(wine_df[wine_df['lda_topic'] == label]['category'].value_counts())
    print()

topic 3 observes original label counts of:
red          503
white         94
champagne      8
sparkling      8
Name: category, dtype: int64

topic 4 observes original label counts of:
red          1341
sparkling      30
champagne      30
white          16
Name: category, dtype: int64

topic 7 observes original label counts of:
red          1068
white           4
sparkling       2
champagne       2
Name: category, dtype: int64

topic 0 observes original label counts of:
red          3822
white        2266
champagne     391
sparkling     391
Name: category, dtype: int64

topic 8 observes original label counts of:
red      719
white      6
Name: category, dtype: int64

topic 2 observes original label counts of:
white        81
champagne    44
sparkling    44
red           7
Name: category, dtype: int64

topic 1 observes original label counts of:
white        146
champagne    104
sparkling    104
red           25
Name: category, dtype: int64

topic 6 observes original label counts of:
red 

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
